In [1]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import ipdb
import re
from datasets import load_dataset
from diversity import compression_ratio, homogenization_score, ngram_diversity_score, extract_patterns, get_pos, pos_patterns, token_patterns
import json

import matplotlib.pyplot as plt
# import mplcursors
import seaborn as sns
%matplotlib inline
sns.set(style='darkgrid', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=2)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('chained_assignment',None)

# Set random seeds for reproducibility on a specific machine
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)
np.set_printoptions(precision=3)

## Dolly human written responses diversity analysis

In [10]:
dolly = load_dataset("databricks/databricks-dolly-15k")["train"].filter(lambda row: row['category']=='creative_writing').to_pandas()
sample = pd.read_csv('../data/dolly_creative_prompts_sample.tsv', sep='\t')

In [11]:
sample.head()

,index,instruction
0,525,Please propose an argument to convince my mother that she increases the amount of money that I get every month during my studies. I think the current amount is too low.
1,172,"Write a paragraph to refute a claim by a colleague that ancient structures such as Stonehenge, the Great Pyramid are evidence of UFO activities on Earth"
2,109,What is the best way to answer an interview question?
3,37,Write the first paragraph of an advertising brochure describing an hotel nearby the Annecy Lake in France
4,295,"Write an intro to a meetup about music, medicine, and machines"


In [12]:
dolly

,instruction,context,response,category
0,write a scene between two actors discussing movie Inception,,"Actress 1: ""Oh my god, have you seen Inception?""\n\nActor 2: ""Yes! That movie blew my mind. What did you think of it?""\n\nActress 1: ""Honestly, I was so confused for most of it. But it was so interesting, I couldn't look away. I had to know what was going on.""\n\nActor 2: ""That's exactly how I felt. It's one of those movies that you have to pay attention to every detail, or you'll get lost.""\n\nActress 1: ""Absolutely. And the way they played with time and reality was so cool. I don't think I've ever seen a movie like that before.""\n\nActor 2: ""Yeah, the concept of entering someone's dreams to implant an idea was mind-boggling. And the way they executed it was so well done.""\n\nActress 1: ""It was. And the cast was amazing too. Leonardo DiCaprio was incredible as usual, and I loved seeing Ellen Page in a different type of role.""\n\nActor 2: ""Definitely. And the action scenes were so intense. The zero-gravity hallway fight was insane.""\n\nActress 1: ""Oh my god, yes. I was on the edge of my seat for that one. And the ending...I still can't decide what it means.""\n\nActor 2: ""I know, right? It's one of those movies that leaves you with so many questions. But that's part of what makes it so great.""\n\nActress 1: ""Exactly. I love when movies challenge me to think and leave me wanting more. Inception definitely did that.""\n\nActor 2: ""Agreed. It's definitely one of my all-time favorites. I could watch it a hundred times and still discover something new.""\n\nActress 1: ""Same here. It's just one of those movies that sticks with you long after it's over.""",creative_writing
1,Write a short paragraph about why you should not have both a pet cat and a pet bird.,,"Having both a pet bird and a pet cat is not a good idea because cats will try to hunt birds, which are naturally the prey of cats. This hunting instinct is completely normal, and it isn't fair to the cat to try to train it not to attack birds, as it is simply in its nature to do so. However, letting a cat hunt your pet bird is similarly not humane, as it will be very stressful to the bird and could lead to the bird being injured. Larger birds are also not a good idea, as the bird could cause injury to the cat if the cat tries to attack it. Overall, it just isn't a good idea to have both cats and birds, since they tend not to coexist well, and pet owners should just pick one species to have.",creative_writing
2,Write a short story about a person who discovers a hidden room in their house. The story should include a plot twist and a clear resolution at the end.,,"It was going to be a long day. With her brother-in-law out of town, Lisa had agreed to watch her niece Stella for the afternoon and the little girl was known to be a bit of a handful. Her plan had been to spend the afternoon at the park to let Stella run and climb, to wear her out a bit. Unfortunately, this outing was foiled by the constant downpour of rain so Lisa resigned herself to a day inside with the tiny tyrant. Stella arrived early and excited to play. Lisa decided to begin with a game of hide and seek, after all this could take up a good part of the morning without too much trouble. Stella ran off to hide and Lisa sat quietly drinking coffee while calling out to the little girl, pretending to seek. Eventually she set off to find the little girl but soon she realized that something was wrong. No matter where she looked, Stella was nowhere to be found. The house wasn't that large, making her disappearance even more perplexing. Lisa had only recently moved into the home that she bought from her great aunt shortly before her death and she hasn't even moved out all of the existing furniture. Maybe Stella had gotten stuck somewhere behind the large wardrobe in the bedroom or she had climbed up into a cupboard in the laundry room. Panic was starting to settle in as she moved the wardrobe further 

In [18]:
sample['response'] = sample['index'].apply(lambda x: dolly.loc[x, 'response'])
sample.head()

,index,instruction,response
0,525,Please propose an argument to convince my mother that she increases the amount of money that I get every month during my studies. I think the current amount is too low.,"Dear Mum, I would like to talk to you about the monthly money I get for my studies. I am very grateful that you support me financially during my studies. At the same time, I feel that the current amount is a little bit too low in order to sustain myself. I estimated my monthly expenses and it seems like the money is not enough. We can go through the details if you want. I want to ask you if you can increase the amount so that I can cover my expenses. I understand if you cannot do this or have other reasons against this. However, I want you to know that if you do not increase my monthly money, I will be forced to work part-time next to my studies. There is research that shows that students who work part-time have worse grades than people who do not have to work part-time to afford their studies. I would love to dedicate my full time to my studies to get the best grades and learning outcomes possible; thus, I would prefer not to work part-time. What do you think?"
1,172,"Write a paragraph to refute a claim by a colleague that ancient structures such as Stonehenge, the Great Pyramid are evidence of UFO activities on Earth","There are several strong arguments against concluding that alien races built the structures like the Stonehenge or the Pyramids. First, there is evidence that these magnificent structures were built by humans. For example, tombs of the pyramid builders were found in the vicinity of the Great Pyramind (with human remains!). The technology to move and assemble the massive stones of the Stonehenge has been shown to be available to humans at that time. Second, given that space is vast the chance of finding one tiny planet among billions is infinitesimal. If alien intelligence had indeed found our one inhabited planet they are unlikely to have just created couple of structures and avoided any further contact with Earth and its inhabitants. In science, Occam’s Razor (“the best explanation is the simplest one”) is a guiding principle and until there is much more evidence of alien visitation it would be inadvisable to jump to a conclusion that aliens built these structures."
2,109,What is the best way to answer an interview question?,"The first recommended step is to ask clarifying questions to ensure you understand the question correctly and gather the requirements needed to answer effectively. If it is a mathematical or computer science question, go ahead and work your way through the problem while you verbalize your thought process along the way. If it is a situation question, you should use the STAR method. 1. Situation - set the scene and provide context, 2. Task - describe what was needed to be done and why, 3. Action - explain what action you took, and 4. Result - describe the result of your actions and what you learned from the experience"
3,37,Write the first paragraph of an advertising brochure describing an hotel nearby the Annecy Lake in France,"This fantastic hotel is ideally located, minutes away from the Annecy Lake, one of the purest in Europe. The water temperature reaches 27 degrees celcius at the peak season, allowing the guests to swim and enjoy several water activities including wake surf and pedalo. Finally, this region is known for being the home of several cheese specialties like ""Raclette"" and ""Tartiflette"" to name a few."
4,295,"Write an intro to a meetup about music, medicine, and machines","Music, Medicine, and Machines\n\nJoin us for an imaginative, authentic, and offbeat networking experience that will advance your knowledge of the technologies that impact the health and well-being of our human condition. To make things interesting (but keep things real), our meetup incorporates an element of music to remind us, first and foremost, we want to advance technology for the greater good, b

In [20]:
human_responses = sample['response'].values.tolist()

In [22]:
cr = compression_ratio(human_responses, 'gzip')
nds = ngram_diversity_score(human_responses, 4)
print(f"CR: {cr}, NDS: {nds}")

CR: 2.508, NDS: 3.026


In [23]:
joined_pos, tuples = get_pos(human_responses)

In [26]:
ngrams_pos = token_patterns(joined_pos, 5, 10)
ngrams_pos

[('DT NN IN DT NN', 53),
 ('NN IN DT JJ NN', 49),
 ('IN DT NN IN DT', 43),
 ('DT JJ NN IN DT', 43),
 ('IN DT JJ NN IN', 33),
 ('IN DT JJ NN .', 32),
 ('JJ NN IN DT NN', 30),
 ('IN DT NN IN NN', 28),
 ('VB DT NN IN DT', 26),
 ('NN IN DT NN IN', 25)]

In [28]:
cr = compression_ratio(joined_pos, 'gzip')
cr

4.908

## Llama model diversity

In [51]:
llama8b = pd.read_csv('../output/Llama-3.1-8B-Instruct_dolly_output.tsv', sep='\t')

In [52]:
persona_ids = []
prompt_ids = []
for persona_id in range(100):
    for prompt_id in range(100):
        persona_ids += [persona_id]*3
        prompt_ids += [prompt_id]*3

len(persona_ids), len(prompt_ids)

(30000, 30000)

In [53]:
llama8b['persona_id'] = persona_ids
llama8b['prompt_id'] = prompt_ids
llama8b.head()

,prompt,response,persona_id,prompt_id
0,Assume you are the following persona: a PR manager with insights into public relations strategy and press releases for influencers.\n\nNow respond to the following question/instruction appropriately from the perspective of the above persona:\n\nPlease propose an argument to convince my mother that she increases the amount of money that I get every month during my studies. I think the current amount is too low.,"\n\nAs a PR manager, I'd love to help you craft a compelling argument to present to your mother. Here's a suggested approach:\n\n**Emphasize your commitment to your studies and future goals**\n\nStart by expressing your gratitude for your mother's current support and reassure her that you're committed to your studies and future goals. Highlight your academic achievements, any notable projects or extracurricular activities you've been involved in, and your plans for after graduation.\n\n**Provide a breakdown of your expenses**\n\nCreate a detailed breakdown of your monthly expenses, including tuition fees, textbooks, living costs, transportation, and any other relevant expenses. This will help your mother understand where the money is going and why you need a higher allowance. Be sure to include any potential increases in expenses, such as rent or utility costs.\n\n**Highlight the benefits of a higher allowance**\n\nExplain how a higher allowance will benefit you in the short and long term. For example, you may be able to:\n\n* Focus more on your studies and less on part-time jobs, leading to better academic performance\n* Invest in personal development opportunities, such as courses or workshops, to enhance your skills and career prospects\n* Build an emergency fund to cover unexpected expenses or financial setbacks\n* Develop financial literacy and responsibility",0,0
1,Assume you are the following persona: a PR manager with insights into public relations strategy and press releases for influencers.\n\nNow respond to the following question/instruction appropriately from the perspective of the above persona:\n\nPlease propose an argument to convince my mother that she increases the amount of money that I get every month during my studies. I think the current amount is too low.,"\n\nAs a PR manager, I'd love to help you craft a compelling argument to persuade your mother to increase your monthly allowance. Here's a suggested approach:\n\n**Start with empathy and understanding**\n\n Begin by acknowledging your mother's concerns and values. Express your appreciation for her support and the importance she places on your education. This will help set a positive tone for the conversation.\n\n**Highlight your achievements and responsibilities**\n\n Emphasize your academic performance, any part-time jobs you've taken on, or extracurricular activities you're involved in. This demonstrates your commitment to your studies and your ability to manage your finances responsibly. Mention any specific expenses you've incurred due to your academic pursuits, such as textbooks, transportation, or equipment.\n\n**Present a clear and realistic budget**\n\n Create a breakdown of your monthly expenses, including essential costs like food, rent, utilities, and entertainment. Be sure to include any specific financial goals you have, such as saving for a car or a laptop. This will help your mother understand your financial situation and the need for a higher allowance.\n\n**Propose a solution, not a demand**\n\n Instead of simply asking for a specific amount, suggest a range or a percentage increase. For example, you could propose a 10-20% increase in your",0,0
2,Assume you are the following persona: a PR manager with insights into public relations strategy and press releases for influencers.\n\nNow respond to the following question/instruction appropriately from the perspective of the above persona:\n\nPlease propose an argument to convince my mother that she increases the amount of money that I get every 

In [73]:
crs = []
ndss = []
for persona_id in tqdm(range(100)):
    ## Take first sample
    responses = llama8b.loc[llama8b.persona_id==persona_id].drop_duplicates(subset=['prompt_id'], keep='first')['response'].values.tolist()
    cr = compression_ratio(responses, 'gzip')
    nds = ngram_diversity_score(responses, 4)
    crs.append(cr)
    ndss.append(nds)
print(f"CR: {np.round(np.mean(cr),2)} +- {np.round(np.std(crs),2)}, NDS: {np.round(np.mean(ndss),2)} +- {np.round(np.std(ndss), 2)}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 51.97it/s]

CR: 2.51 +- 0.1, NDS: 2.79 +- 0.09


In [72]:
crs = []
ndss = []
for persona_id in tqdm(range(100)):
    ## Take last sample
    responses = llama8b.loc[llama8b.persona_id==persona_id].drop_duplicates(subset=['prompt_id'], keep='last')['response'].values.tolist()
    cr = compression_ratio(responses, 'gzip')
    nds = ngram_diversity_score(responses, 4)
    crs.append(cr)
    ndss.append(nds)
print(f"CR: {np.round(np.mean(cr),2)} +- {np.round(np.std(crs),2)}, NDS: {np.round(np.mean(ndss),2)} +- {np.round(np.std(ndss), 2)}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 49.55it/s]

CR: 2.48 +- 0.1, NDS: 2.79 +- 0.09


In [74]:
crs = []
ndss = []
for persona_id in tqdm(range(100)):
    ## Take middle sample of three
    responses = llama8b.loc[llama8b.persona_id==persona_id].groupby(['prompt_id']).tail(2).groupby(['prompt_id']).head(1)['response'].values.tolist()
    cr = compression_ratio(responses, 'gzip')
    nds = ngram_diversity_score(responses, 4)
    crs.append(cr)
    ndss.append(nds)
print(f"CR: {np.round(np.mean(cr),2)} +- {np.round(np.std(crs),2)}, NDS: {np.round(np.mean(ndss),2)} +- {np.round(np.std(ndss), 2)}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 44.11it/s]

CR: 2.51 +- 0.1, NDS: 2.79 +- 0.09


In [75]:
crs_pos = []
for persona_id in tqdm(range(100)):
    responses = llama8b.loc[llama8b.persona_id==persona_id].drop_duplicates(subset=['prompt_id'])['response'].values.tolist()
    joined_pos, tuples = get_pos(responses)
    cr = compression_ratio(joined_pos, 'gzip')
    crs_pos.append(cr)
print(f"CR-POS: {np.round(np.mean(cr),2)} +- {np.round(np.std(crs),2)}")

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [04:59<00:00,  2.99s/it]

CR-POS: 5.14 +- 0.1


## See examples of most common POS patterns

In [76]:
unique_responses = llama8b.drop_duplicates(subset=['persona_id', 'prompt_id'])['response'].values.tolist()

n = 5 
top_n = 100
patterns = extract_patterns(unique_responses, n, top_n)

In [77]:
patterns

{'DT NN IN DT NN': {'a diamond in the eye',
  'a master of the drill',
  'the weight of the universe',
  'the middle of the night',
  'the absurdity of a system',
  'the cream of the crop',
  'the likelihood of an attack',
  'a copy of an object',
  'the probability of an event',
  'the behavior of the dog',
  'the land of the midnight',
  'the pinnacle of the sport',
  'the content of this song',
  'a hallmark As a game',
  'a feel for the lay',
  'the midst of a match',
  'the din of the coffee',
  'the star of the show',
  'a cuppa on the hotel',
  'a reference to the concept',
  'a reflection of the soul',
  'the complexity of the issue',
  'the rhythm of the flow',
  'The energy in the stadium',
  'A reflection of the world',
  'the heart of this phenomenon',
  'the heart of a quaint',
  'the size of the maze',
  'the midst of an interview',
  'a reconnaissance of the city',
  'a departure from the fantasy',
  'a spacesuit with a life',
  'a parent to a child',
  'the face of a gr

In [ ]:
len_dict = {'human': [len(x) for x in human_responses], 'llama8b': [len(x) for x in unique_responses]}
sns.kdeplot(len_dict)

## llama-1b

In [44]:
llama1b = pd.read_csv('../output/Llama-3.2-1B-Instruct_dolly_output.tsv', sep='\t')
llama1b['persona_id'] = persona_ids
llama1b['prompt_id'] = prompt_ids

In [45]:
unique_ll_responses = llama1b.drop_duplicates(subset=['persona_id', 'prompt_id'])['response'].values.tolist()

In [46]:
cr = compression_ratio(unique_ll_responses, 'gzip')
nds = ngram_diversity_score(unique_ll_responses, 4)
print(f"CR: {cr}, NDS: {nds}")

CR: 2.768, NDS: 1.701


In [48]:
joined_pos, tuples = get_pos(unique_ll_responses)
ngrams_pos = token_patterns(joined_pos, 5, 10)
ngrams_pos

[('DT NN IN DT NN', 4731),
 ('DT JJ NN IN DT', 4469),
 ('NN IN DT JJ NN', 4005),
 ('NN NN NN NN NN', 3633),
 ('IN DT JJ NN ,', 3555),
 ('IN DT JJ NN IN', 3479),
 ('JJ NN IN DT NN', 3139),
 ('IN DT NN IN DT', 3107),
 ('DT JJ NN IN NN', 2976),
 ('NN , PRP MD VB', 2942)]

In [49]:
cr = compression_ratio(joined_pos, 'gzip')
cr

5.537